In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Import simulation functions for comparison
from simulation.engine.sparam_to_ew import snp_eyewidth_simulation
from simulation.parameters.bound_param import ParameterSet

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:
# Configure the path to your pickle files
# Adjust this path based on your actual output directory structure
pickle_dir = Path("./data/training_data")  # Update this path as needed

# Find all pickle files
pickle_files = list(pickle_dir.rglob("*.pkl"))
print(f"Found {len(pickle_files)} pickle files")

# Show first few files
for i, pfile in enumerate(pickle_files[:5]):
    print(f"{i+1}. {pfile.relative_to(pickle_dir)}")

if len(pickle_files) > 5:
    print(f"... and {len(pickle_files) - 5} more files")


In [ ]:
# Load and examine the first pickle file
if pickle_files:
    sample_file = pickle_files[0]
    print(f"Examining: {sample_file.name}\n")
    
    with open(sample_file, 'rb') as f:
        sample_data = pickle.load(f)
    
    print("Data structure:")
    for key, value in sample_data.items():
        if isinstance(value, list):
            print(f"  {key}: list with {len(value)} items")
            if len(value) > 0:
                print(f"    First item type: {type(value[0])}")
                if key == 'configs' and len(value) > 0:
                    print(f"    Config length: {len(value[0])} parameters")
                elif key == 'line_ews' and len(value) > 0:
                    print(f"    Line EW shape: {np.array(value[0]).shape}")
        else:
            print(f"  {key}: {type(value)}")
else:
    print("No pickle files found. Please check the pickle_dir path.")


In [ ]:
if pickle_files and sample_data:
    print("Detailed data examination:")
    print("=" * 50)
    
    # Examine configs
    if sample_data['configs']:
        config_array = np.array(sample_data['configs'])
        print(f"\nConfigs:")
        print(f"  Shape: {config_array.shape}")
        print(f"  Min values: {config_array.min(axis=0)[:10]}...")
        print(f"  Max values: {config_array.max(axis=0)[:10]}...")
        print(f"  Mean values: {config_array.mean(axis=0)[:10]}...")
    
    # Examine line eye widths
    if sample_data['line_ews']:
        line_ews_array = np.array(sample_data['line_ews'])
        print(f"\nLine Eye Widths:")
        print(f"  Shape: {line_ews_array.shape}")
        print(f"  Min: {line_ews_array.min():.3f}")
        print(f"  Max: {line_ews_array.max():.3f}")
        print(f"  Mean: {line_ews_array.mean():.3f}")
        print(f"  Std: {line_ews_array.std():.3f}")
        print(f"  Closed eyes (EW < 0): {(line_ews_array < 0).sum()} / {line_ews_array.size}")
    
    # Examine SNP files
    if sample_data['snp_txs']:
        unique_tx = set(sample_data['snp_txs'])
        unique_rx = set(sample_data['snp_rxs'])
        print(f"\nSNP Files:")
        print(f"  Unique TX files: {len(unique_tx)}")
        print(f"  Unique RX files: {len(unique_rx)}")
        print(f"  Sample TX: {list(unique_tx)[0] if unique_tx else 'None'}")
        print(f"  Sample RX: {list(unique_rx)[0] if unique_rx else 'None'}")
    
    # Examine directions
    if sample_data['directions']:
        directions_array = np.array(sample_data['directions'])
        print(f"\nDirections:")
        print(f"  Shape: {directions_array.shape}")
        if directions_array.size > 0:
            print(f"  Unique patterns: {len(set(tuple(row) for row in directions_array))}")
            print(f"  Sample directions: {directions_array[0]}")


In [ ]:
# Aggregate data from all pickle files
all_configs = []
all_line_ews = []
all_directions = []
file_stats = []

print("Loading all pickle files...")
for pfile in pickle_files:
    try:
        with open(pfile, 'rb') as f:
            data = pickle.load(f)
        
        n_samples = len(data.get('configs', []))
        file_stats.append({
            'file': pfile.name,
            'samples': n_samples,
            'trace_name': pfile.stem
        })
        
        if data.get('configs'):
            all_configs.extend(data['configs'])
        if data.get('line_ews'):
            all_line_ews.extend(data['line_ews'])
        if data.get('directions'):
            all_directions.extend(data['directions'])
            
    except Exception as e:
        print(f"Error loading {pfile.name}: {e}")

print(f"\nLoaded data from {len(file_stats)} files")
print(f"Total samples: {len(all_configs)}")
print(f"Total eye width measurements: {len(all_line_ews)}")


In [ ]:
# Create summary statistics DataFrame
summary_df = pd.DataFrame(file_stats)
summary_df = summary_df.sort_values('samples', ascending=False)

print("File-wise sample counts:")
print(summary_df.head(10))
print(f"\nTotal files: {len(summary_df)}")
print(f"Total samples: {summary_df['samples'].sum()}")
print(f"Average samples per file: {summary_df['samples'].mean():.1f}")
print(f"Min samples: {summary_df['samples'].min()}")
print(f"Max samples: {summary_df['samples'].max()}")


In [ ]:
if all_line_ews:
    # Convert to numpy array for analysis
    line_ews_array = np.array(all_line_ews)
    
    print(f"Eye Width Analysis:")
    print(f"  Total measurements: {line_ews_array.size}")
    print(f"  Shape: {line_ews_array.shape}")
    print(f"  Min: {line_ews_array.min():.3f}")
    print(f"  Max: {line_ews_array.max():.3f}")
    print(f"  Mean: {line_ews_array.mean():.3f}")
    print(f"  Median: {np.median(line_ews_array):.3f}")
    print(f"  Std: {line_ews_array.std():.3f}")
    
    # Analyze closed eyes
    closed_eyes = line_ews_array < 0
    print(f"  Closed eyes (EW < 0): {closed_eyes.sum()} ({closed_eyes.mean()*100:.1f}%)")
    
    # Analyze very wide eyes (might be artifacts)
    wide_eyes = line_ews_array > 95
    print(f"  Very wide eyes (EW > 95): {wide_eyes.sum()} ({wide_eyes.mean()*100:.1f}%)")
    
    # Plot distributions
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Overall distribution
    axes[0,0].hist(line_ews_array.flatten(), bins=50, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('Overall Eye Width Distribution')
    axes[0,0].set_xlabel('Eye Width')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].axvline(line_ews_array.mean(), color='red', linestyle='--', label=f'Mean: {line_ews_array.mean():.2f}')
    axes[0,0].legend()
    
    # Distribution excluding closed eyes
    open_eyes = line_ews_array[line_ews_array >= 0]
    if len(open_eyes) > 0:
        axes[0,1].hist(open_eyes.flatten(), bins=50, alpha=0.7, edgecolor='black', color='green')
        axes[0,1].set_title('Eye Width Distribution (Open Eyes Only)')
        axes[0,1].set_xlabel('Eye Width')
        axes[0,1].set_ylabel('Frequency')
        axes[0,1].axvline(open_eyes.mean(), color='red', linestyle='--', label=f'Mean: {open_eyes.mean():.2f}')
        axes[0,1].legend()
    
    # Box plot by line (if multiple lines)
    if line_ews_array.ndim > 1 and line_ews_array.shape[1] > 1:
        line_data = [line_ews_array[:, i] for i in range(line_ews_array.shape[1])]
        axes[1,0].boxplot(line_data, labels=[f'Line {i}' for i in range(len(line_data))])
        axes[1,0].set_title('Eye Width Distribution by Line')
        axes[1,0].set_ylabel('Eye Width')
    else:
        axes[1,0].text(0.5, 0.5, 'Single line data or 1D array', ha='center', va='center', transform=axes[1,0].transAxes)
        axes[1,0].set_title('Eye Width by Line (N/A)')
    
    # Cumulative distribution
    sorted_ews = np.sort(line_ews_array.flatten())
    y_vals = np.arange(1, len(sorted_ews) + 1) / len(sorted_ews)
    axes[1,1].plot(sorted_ews, y_vals)
    axes[1,1].set_title('Cumulative Distribution of Eye Widths')
    axes[1,1].set_xlabel('Eye Width')
    axes[1,1].set_ylabel('Cumulative Probability')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No eye width data found in pickle files.")


In [ ]:
print("Data Quality Checks:")
print("=" * 30)

# Check for consistent data lengths
inconsistent_files = []
for pfile in pickle_files:
    try:
        with open(pfile, 'rb') as f:
            data = pickle.load(f)
        
        lengths = {key: len(val) for key, val in data.items() if isinstance(val, list)}
        if len(set(lengths.values())) > 1:
            inconsistent_files.append((pfile.name, lengths))
    except Exception as e:
        print(f"Error checking {pfile.name}: {e}")

if inconsistent_files:
    print(f"\n⚠️  Found {len(inconsistent_files)} files with inconsistent data lengths:")
    for fname, lengths in inconsistent_files:
        print(f"  {fname}: {lengths}")
else:
    print("✅ All files have consistent data lengths")

# Check for missing or corrupted data
if all_line_ews:
    line_ews_array = np.array(all_line_ews)
    nan_count = np.isnan(line_ews_array).sum()
    inf_count = np.isinf(line_ews_array).sum()
    
    print(f"\nData integrity:")
    print(f"  NaN values: {nan_count}")
    print(f"  Infinite values: {inf_count}")
    print(f"  Values < -1 (suspicious): {(line_ews_array < -1).sum()}")
    print(f"  Values > 100 (suspicious): {(line_ews_array > 100).sum()}")

# File size analysis
file_sizes = [pfile.stat().st_size / 1024 for pfile in pickle_files]  # KB
if file_sizes:
    print(f"\nFile sizes:")
    print(f"  Average: {np.mean(file_sizes):.1f} KB")
    print(f"  Min: {np.min(file_sizes):.1f} KB")
    print(f"  Max: {np.max(file_sizes):.1f} KB")
    print(f"  Total: {np.sum(file_sizes):.1f} KB ({np.sum(file_sizes)/1024:.1f} MB)")


In [ ]:
# Generate and save summary report
report = []
report.append("EYE DIAGRAM TRAINING DATA SUMMARY REPORT")
report.append("=" * 50)
report.append(f"Generated: {pd.Timestamp.now()}")
report.append(f"Data directory: {pickle_dir}")
report.append("")

# Dataset overview
report.append("DATASET OVERVIEW:")
report.append(f"  Total pickle files: {len(pickle_files)}")
report.append(f"  Total samples: {len(all_configs)}")
if len(all_configs) > 0:
    report.append(f"  Parameters per sample: {len(all_configs[0])}")
if len(all_line_ews) > 0:
    line_ews_array = np.array(all_line_ews)
    report.append(f"  Lines per sample: {line_ews_array.shape[1] if line_ews_array.ndim > 1 else 1}")

report.append("")

# Eye width statistics
if len(all_line_ews) > 0:
    line_ews_array = np.array(all_line_ews)
    report.append("EYE WIDTH STATISTICS:")
    report.append(f"  Mean: {line_ews_array.mean():.3f}")
    report.append(f"  Std: {line_ews_array.std():.3f}")
    report.append(f"  Min: {line_ews_array.min():.3f}")
    report.append(f"  Max: {line_ews_array.max():.3f}")
    report.append(f"  Closed eyes: {(line_ews_array < 0).sum()} ({(line_ews_array < 0).mean()*100:.1f}%)")
    report.append("")

# File statistics
if len(file_stats) > 0:
    summary_df = pd.DataFrame(file_stats)
    report.append("FILE STATISTICS:")
    report.append(f"  Average samples per file: {summary_df['samples'].mean():.1f}")
    report.append(f"  Min samples per file: {summary_df['samples'].min()}")
    report.append(f"  Max samples per file: {summary_df['samples'].max()}")
    report.append("")

# Print and save report
report_text = "\n".join(report)
print(report_text)

# Save to file
report_file = Path("training_data_summary.txt")
with open(report_file, 'w') as f:
    f.write(report_text)
    
print(f"\nReport saved to: {report_file}")


In [ ]:
# Select a subset of files for validation (to avoid long computation)
validation_files = pickle_files[:min(3, len(pickle_files))]  # Validate first 3 files
print(f"Validating {len(validation_files)} files...")

validation_results = []
comparison_data = {
    'pickle_ews': [],
    'fresh_ews': [],
    'differences': [],
    'relative_errors': [],
    'file_names': [],
    'sample_indices': []
}

for file_idx, pfile in enumerate(validation_files):
    print(f"\nValidating {pfile.name}...")
    
    try:
        # Load pickle data
        with open(pfile, 'rb') as f:
            data = pickle.load(f)
        
        n_samples = len(data.get('configs', []))
        print(f"  Found {n_samples} samples")
        
        # Validate a subset of samples from each file (to manage computation time)
        sample_indices = list(range(0, min(5, n_samples), max(1, n_samples//5)))  # At most 5 samples per file
        
        for i, sample_idx in enumerate(sample_indices):
            print(f"  Validating sample {sample_idx+1}/{n_samples}...")
            
            # Extract data for this sample
            config_list = data['configs'][sample_idx]
            pickle_ew = np.array(data['line_ews'][sample_idx])
            directions = np.array(data['directions'][sample_idx]) if data['directions'][sample_idx] else None
            
            # Get SNP file paths
            snp_horiz = Path(data.get('snp_horiz', [''])[sample_idx] if isinstance(data.get('snp_horiz', []), list) else pfile.parent / f"{pfile.stem}.s4p")
            snp_tx = Path(data['snp_txs'][sample_idx])
            snp_rx = Path(data['snp_rxs'][sample_idx])
            
            # Reconstruct config object (assuming it follows ParameterSet structure)
            # Note: This might need adjustment based on your actual ParameterSet implementation
            config = ParameterSet.from_list(config_list)
            
            try:
                # Run fresh simulation
                result = snp_eyewidth_simulation(
                    config=config,
                    snp_files=(snp_horiz, snp_tx, snp_rx),
                    directions=directions,
                    device="cuda"  # or "cpu" based on your preference
                )
                
                # Handle return format (might be tuple or just line_ew)
                if isinstance(result, tuple):
                    fresh_ew, fresh_directions = result
                else:
                    fresh_ew = result
                    fresh_directions = directions
                
                fresh_ew = np.array(fresh_ew)
                
                # Calculate differences
                diff = fresh_ew - pickle_ew
                rel_error = np.abs(diff) / (np.abs(pickle_ew) + 1e-10)  # Avoid division by zero
                
                # Store comparison data
                comparison_data['pickle_ews'].extend(pickle_ew.flatten())
                comparison_data['fresh_ews'].extend(fresh_ew.flatten())
                comparison_data['differences'].extend(diff.flatten())
                comparison_data['relative_errors'].extend(rel_error.flatten())
                comparison_data['file_names'].extend([pfile.name] * len(pickle_ew.flatten()))
                comparison_data['sample_indices'].extend([sample_idx] * len(pickle_ew.flatten()))
                
                # Print sample comparison
                print(f"    Pickle EW: {pickle_ew}")
                print(f"    Fresh EW:  {fresh_ew}")
                print(f"    Difference: {diff}")
                print(f"    Max abs diff: {np.abs(diff).max():.6f}")
                print(f"    Max rel error: {rel_error.max():.6f}")
                
            except Exception as e:
                print(f"    Error in fresh simulation: {e}")
                continue
                
    except Exception as e:
        print(f"Error processing {pfile.name}: {e}")
        continue

print(f"\nValidation completed!")
print(f"Total comparisons: {len(comparison_data['differences'])}")

if len(comparison_data['differences']) > 0:
    differences = np.array(comparison_data['differences'])
    rel_errors = np.array(comparison_data['relative_errors'])
    
    print(f"\nOverall Validation Statistics:")
    print(f"  Mean absolute difference: {np.abs(differences).mean():.6f}")
    print(f"  Max absolute difference: {np.abs(differences).max():.6f}")
    print(f"  RMS difference: {np.sqrt((differences**2).mean()):.6f}")
    print(f"  Mean relative error: {rel_errors.mean():.6f}")
    print(f"  Max relative error: {rel_errors.max():.6f}")
    print(f"  Differences > 1e-3: {(np.abs(differences) > 1e-3).sum()}")
    print(f"  Relative errors > 1%: {(rel_errors > 0.01).sum()}")


In [ ]:
# Visualization of comparison results
if len(comparison_data['differences']) > 0:
    pickle_ews = np.array(comparison_data['pickle_ews'])
    fresh_ews = np.array(comparison_data['fresh_ews'])
    differences = np.array(comparison_data['differences'])
    rel_errors = np.array(comparison_data['relative_errors'])
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Scatter plot: Pickle vs Fresh
    axes[0,0].scatter(pickle_ews, fresh_ews, alpha=0.6, s=20)
    axes[0,0].plot([pickle_ews.min(), pickle_ews.max()], [pickle_ews.min(), pickle_ews.max()], 'r--', label='Perfect match')
    axes[0,0].set_xlabel('Pickle Eye Width')
    axes[0,0].set_ylabel('Fresh Eye Width')
    axes[0,0].set_title('Pickle vs Fresh Eye Width')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Histogram of differences
    axes[0,1].hist(differences, bins=30, alpha=0.7, edgecolor='black')
    axes[0,1].axvline(0, color='red', linestyle='--', label='Zero difference')
    axes[0,1].axvline(differences.mean(), color='orange', linestyle='--', label=f'Mean: {differences.mean():.6f}')
    axes[0,1].set_xlabel('Difference (Fresh - Pickle)')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].set_title('Distribution of Differences')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Histogram of relative errors
    axes[0,2].hist(rel_errors, bins=30, alpha=0.7, edgecolor='black', color='green')
    axes[0,2].axvline(rel_errors.mean(), color='red', linestyle='--', label=f'Mean: {rel_errors.mean():.6f}')
    axes[0,2].set_xlabel('Relative Error')
    axes[0,2].set_ylabel('Frequency')
    axes[0,2].set_title('Distribution of Relative Errors')
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Absolute differences vs eye width magnitude
    axes[1,0].scatter(pickle_ews, np.abs(differences), alpha=0.6, s=20)
    axes[1,0].set_xlabel('Pickle Eye Width')
    axes[1,0].set_ylabel('|Difference|')
    axes[1,0].set_title('Absolute Difference vs Eye Width')
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Relative errors vs eye width magnitude  
    axes[1,1].scatter(pickle_ews, rel_errors, alpha=0.6, s=20, color='green')
    axes[1,1].set_xlabel('Pickle Eye Width')
    axes[1,1].set_ylabel('Relative Error')
    axes[1,1].set_title('Relative Error vs Eye Width')
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Box plot of differences by file
    file_names = comparison_data['file_names']
    unique_files = list(set(file_names))
    if len(unique_files) > 1:
        diff_by_file = [differences[np.array(file_names) == fname] for fname in unique_files]
        axes[1,2].boxplot(diff_by_file, labels=[fname[:10] + '...' if len(fname) > 10 else fname for fname in unique_files])
        axes[1,2].set_ylabel('Difference')
        axes[1,2].set_title('Differences by File')
        axes[1,2].tick_params(axis='x', rotation=45)
    else:
        axes[1,2].text(0.5, 0.5, 'Only one file\nvalidated', ha='center', va='center', transform=axes[1,2].transAxes)
        axes[1,2].set_title('Differences by File (N/A)')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics table
    summary_stats = pd.DataFrame({
        'Metric': ['Mean Abs Diff', 'Max Abs Diff', 'RMS Diff', 'Mean Rel Error', 'Max Rel Error', '# Large Diffs (>1e-3)', '# Large Rel Errors (>1%)'],
        'Value': [
            f"{np.abs(differences).mean():.6f}",
            f"{np.abs(differences).max():.6f}", 
            f"{np.sqrt((differences**2).mean()):.6f}",
            f"{rel_errors.mean():.6f}",
            f"{rel_errors.max():.6f}",
            f"{(np.abs(differences) > 1e-3).sum()} / {len(differences)}",
            f"{(rel_errors > 0.01).sum()} / {len(rel_errors)}"
        ]
    })
    
    print("\nValidation Summary Statistics:")
    print(summary_stats.to_string(index=False))
    
else:
    print("No validation data available for plotting.")


In [ ]:
# Identify and analyze samples with large differences
if len(comparison_data['differences']) > 0:
    differences = np.array(comparison_data['differences'])
    rel_errors = np.array(comparison_data['relative_errors'])
    pickle_ews = np.array(comparison_data['pickle_ews'])
    fresh_ews = np.array(comparison_data['fresh_ews'])
    
    # Find samples with large absolute differences
    large_abs_diff_threshold = np.percentile(np.abs(differences), 95)  # Top 5% of differences
    large_abs_diff_mask = np.abs(differences) > large_abs_diff_threshold
    
    # Find samples with large relative errors (excluding very small eye widths)
    significant_ew_mask = np.abs(pickle_ews) > 1.0  # Only consider significant eye widths
    large_rel_error_threshold = np.percentile(rel_errors[significant_ew_mask], 95) if significant_ew_mask.any() else 0.1
    large_rel_error_mask = (rel_errors > large_rel_error_threshold) & significant_ew_mask
    
    print(f"Analysis of samples with large differences:")
    print(f"  Large absolute difference threshold (95th percentile): {large_abs_diff_threshold:.6f}")
    print(f"  Samples with large absolute differences: {large_abs_diff_mask.sum()}")
    print(f"  Large relative error threshold (95th percentile): {large_rel_error_threshold:.6f}")
    print(f"  Samples with large relative errors: {large_rel_error_mask.sum()}")
    
    # Show worst cases
    worst_abs_diff_indices = np.argsort(np.abs(differences))[-5:]  # 5 worst absolute differences
    worst_rel_error_indices = np.argsort(rel_errors)[-5:]  # 5 worst relative errors
    
    print(f"\nWorst Absolute Differences:")
    for i, idx in enumerate(reversed(worst_abs_diff_indices)):
        print(f"  {i+1}. Pickle: {pickle_ews[idx]:.6f}, Fresh: {fresh_ews[idx]:.6f}, Diff: {differences[idx]:.6f}")
    
    print(f"\nWorst Relative Errors:")
    for i, idx in enumerate(reversed(worst_rel_error_indices)):
        if pickle_ews[idx] != 0:  # Avoid division by zero cases
            print(f"  {i+1}. Pickle: {pickle_ews[idx]:.6f}, Fresh: {fresh_ews[idx]:.6f}, Rel Error: {rel_errors[idx]:.6f}")
    
    # Distribution analysis
    print(f"\nDistribution Analysis:")
    print(f"  Perfect matches (diff = 0): {(differences == 0).sum()}")
    print(f"  Very small diffs (|diff| < 1e-6): {(np.abs(differences) < 1e-6).sum()}")
    print(f"  Small diffs (1e-6 <= |diff| < 1e-3): {((np.abs(differences) >= 1e-6) & (np.abs(differences) < 1e-3)).sum()}")
    print(f"  Medium diffs (1e-3 <= |diff| < 1e-1): {((np.abs(differences) >= 1e-3) & (np.abs(differences) < 1e-1)).sum()}")
    print(f"  Large diffs (|diff| >= 1e-1): {(np.abs(differences) >= 1e-1).sum()}")
    
    # Create a detailed comparison DataFrame for inspection
    detailed_comparison = pd.DataFrame({
        'File': comparison_data['file_names'],
        'Sample_Index': comparison_data['sample_indices'],
        'Pickle_EW': pickle_ews,
        'Fresh_EW': fresh_ews,
        'Difference': differences,
        'Abs_Difference': np.abs(differences),
        'Rel_Error': rel_errors
    })
    
    # Sort by absolute difference for easy inspection
    detailed_comparison = detailed_comparison.sort_values('Abs_Difference', ascending=False)
    
    print(f"\nTop 10 samples with largest differences:")
    print(detailed_comparison.head(10).to_string(index=False, float_format='%.6f'))
    
    # Save detailed comparison to file
    detailed_comparison.to_csv('validation_comparison_details.csv', index=False, float_format='%.8f')
    print(f"\nDetailed comparison saved to: validation_comparison_details.csv")
    
else:
    print("No validation data available for detailed analysis.")


In [ ]:
# Generate comprehensive summary report with validation results
report = []
report.append("EYE DIAGRAM TRAINING DATA SUMMARY REPORT")
report.append("=" * 50)
report.append(f"Generated: {pd.Timestamp.now()}")
report.append(f"Data directory: {pickle_dir}")
report.append("")

# Dataset overview
report.append("DATASET OVERVIEW:")
report.append(f"  Total pickle files: {len(pickle_files)}")
report.append(f"  Total samples: {len(all_configs)}")
if len(all_configs) > 0:
    report.append(f"  Parameters per sample: {len(all_configs[0])}")
if len(all_line_ews) > 0:
    line_ews_array = np.array(all_line_ews)
    report.append(f"  Lines per sample: {line_ews_array.shape[1] if line_ews_array.ndim > 1 else 1}")
report.append("")

# Eye width statistics
if len(all_line_ews) > 0:
    line_ews_array = np.array(all_line_ews)
    report.append("EYE WIDTH STATISTICS:")
    report.append(f"  Mean: {line_ews_array.mean():.3f}")
    report.append(f"  Std: {line_ews_array.std():.3f}")
    report.append(f"  Min: {line_ews_array.min():.3f}")
    report.append(f"  Max: {line_ews_array.max():.3f}")
    report.append(f"  Closed eyes: {(line_ews_array < 0).sum()} ({(line_ews_array < 0).mean()*100:.1f}%)")
    report.append("")

# File statistics
if len(file_stats) > 0:
    summary_df = pd.DataFrame(file_stats)
    report.append("FILE STATISTICS:")
    report.append(f"  Average samples per file: {summary_df['samples'].mean():.1f}")
    report.append(f"  Min samples per file: {summary_df['samples'].min()}")
    report.append(f"  Max samples per file: {summary_df['samples'].max()}")
    report.append("")

# Validation statistics (if validation was performed)
if 'comparison_data' in locals() and len(comparison_data.get('differences', [])) > 0:
    differences = np.array(comparison_data['differences'])
    rel_errors = np.array(comparison_data['relative_errors'])
    report.append("VALIDATION RESULTS (Pickle vs Fresh Simulation):")
    report.append(f"  Files validated: {len([f for f in locals().get('validation_files', [])])}")
    report.append(f"  Samples compared: {len(differences)}")
    report.append(f"  Mean absolute difference: {np.abs(differences).mean():.6f}")
    report.append(f"  Max absolute difference: {np.abs(differences).max():.6f}")
    report.append(f"  RMS difference: {np.sqrt((differences**2).mean()):.6f}")
    report.append(f"  Mean relative error: {rel_errors.mean():.6f}")
    report.append(f"  Max relative error: {rel_errors.max():.6f}")
    report.append(f"  Perfect matches: {(differences == 0).sum()}")
    report.append(f"  Large differences (>1e-3): {(np.abs(differences) > 1e-3).sum()}")
    report.append(f"  Large relative errors (>1%): {(rel_errors > 0.01).sum()}")
    
    # Add interpretation
    if np.abs(differences).max() < 1e-6:
        report.append("  INTERPRETATION: Excellent agreement - differences within numerical precision")
    elif np.abs(differences).max() < 1e-3:
        report.append("  INTERPRETATION: Very good agreement - small numerical differences")
    elif np.abs(differences).max() < 1e-1:
        report.append("  INTERPRETATION: Good agreement - some differences may need investigation")
    else:
        report.append("  INTERPRETATION: Significant differences detected - investigate further")
    report.append("")

# Print and save report
report_text = "\n".join(report)
print(report_text)

# Save to file
report_file = Path("training_data_summary.txt")
with open(report_file, 'w') as f:
    f.write(report_text)
    
print(f"\nReport saved to: {report_file}")
if 'comparison_data' in locals() and len(comparison_data.get('differences', [])) > 0:
    print(f"Validation details saved to: validation_comparison_details.csv")
